<a href="https://colab.research.google.com/github/Bornord/DeepLearning/blob/main/Projet_LeVrai2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROJET CLASSIFICATION

In [ ]:
!git clone https://github.com/Bornord/DeepLearning.git
#path = "./iam/tp3/"

In [ ]:
import matplotlib.pyplot as plt 

def plot_training_analysis(history):
  acc = history.history['accuacy']
  val_acc = history.history['val_accuracy']
  loss = history.history['binary_crossentropy']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.plot(epochs, acc, 'b', linestyle="--",label='Training MAE')
  plt.plot(epochs, val_acc, 'g', label='Validation MAE')
  plt.title('Training and validation MAE')
  plt.legend()

  plt.figure()

  plt.plot(epochs, loss, 'b', linestyle="--",label='Training loss')
  plt.plot(epochs, val_loss,'g', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

from tensorflow.keras.layers import summary
from tensorflow.keras.layers import compile
from tensorflow.keras import optimizers



model = Sequential()
# A COMPLETER
model.add(Conv2D(32, kernel_size= (3,3), strides=(1,1), activation = 'relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, kernel_size= (3,3), strides=(1,1), activation = 'relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(96, kernel_size= (3,3), strides=(1,1), activation = 'relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, kernel_size= (3,3), strides=(1,1), activation = 'relu'))
model.add(MaxPooling2D(2,2))

# ...
model.add(Flatten())    # "Mise à plat" (vectorisation) du tenseur pour permettre de la connecter à une couche dense  
model.add(Dense(512, activation = 'relu'))   # Couche dense, à 512 neurones
model.add(Dense(5, activation = 'sigmoid'))   # Couche de sortie


model.summary()

# AJOUTER EGALEMENT LA FONCTION DE COUT
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(learning_rate=3e-4),
              metrics=['accuracy'])

# TODO
history = model.fit(X_TRAIN, Y_TRAIN, 
                    validation_data=(x_val, y_val),
                    epochs=100,
                    )

